In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/v_dataset.zip .

In [11]:
!ls

drive  sample_data  v_dataset.zip


In [13]:
!unzip v_dataset.zip

Archive:  v_dataset.zip
   creating: v_dataset/
   creating: v_dataset/train/
   creating: v_dataset/train/previous/
  inflating: v_dataset/train/previous/0.jpg  
  inflating: v_dataset/train/previous/1.jpg  
  inflating: v_dataset/train/previous/2.jpg  
  inflating: v_dataset/train/previous/3.jpg  
  inflating: v_dataset/train/previous/4.jpg  
  inflating: v_dataset/train/previous/5.jpg  
  inflating: v_dataset/train/previous/6.jpg  
  inflating: v_dataset/train/previous/7.jpg  
  inflating: v_dataset/train/previous/8.jpg  
  inflating: v_dataset/train/previous/9.jpg  
  inflating: v_dataset/train/previous/10.jpg  
  inflating: v_dataset/train/previous/11.jpg  
  inflating: v_dataset/train/previous/12.jpg  
  inflating: v_dataset/train/previous/13.jpg  
  inflating: v_dataset/train/previous/14.jpg  
  inflating: v_dataset/train/previous/15.jpg  
  inflating: v_dataset/train/previous/16.jpg  
  inflating: v_dataset/train/previous/17.jpg  
  inflating: v_dataset/train/previous/18.jpg  


In [16]:
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 
  
img_width, img_height = 224, 224
  
train_data_dir = 'v_dataset/train'
validation_data_dir = 'v_dataset/valid'
nb_train_samples = 1286
nb_validation_samples = 64
epochs = 50
batch_size = 16
  
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 
    
model = Sequential() 
model.add(Conv2D(32, (2, 2), input_shape = input_shape)) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size =(2, 2))) 
  
model.add(Conv2D(32, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size =(2, 2))) 
  
model.add(Conv2D(64, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size =(2, 2))) 
  
model.add(Flatten()) 
model.add(Dense(64)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(6)) 
model.add(Activation('softmax')) 
print(model.summary())
model.compile(loss ='categorical_crossentropy', 
                     optimizer ='adam', 
                   metrics =['accuracy']) 
  
train_datagen = ImageDataGenerator( 
                rescale = 1. / 255, 
                 shear_range = 0.2, 
                  zoom_range = 0.2, 
            horizontal_flip = True) 
  
test_datagen = ImageDataGenerator(rescale = 1. / 255) 
  
train_generator = train_datagen.flow_from_directory(train_data_dir, 
                              target_size =(img_width, img_height), 
                     batch_size = batch_size, class_mode ='categorical') 
  
validation_generator = test_datagen.flow_from_directory( 
                                    validation_data_dir, 
                   target_size =(img_width, img_height), 
          batch_size = batch_size, class_mode ='categorical') 
# print(train_generator.__dict__)
model.fit_generator(train_generator, 
    steps_per_epoch = nb_train_samples // batch_size, 
    epochs = epochs, validation_data = validation_generator, 
    validation_steps = nb_validation_samples // batch_size) 
  
model.save_weights('model_saved.h5') 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 223, 223, 32)      416       
_________________________________________________________________
activation_6 (Activation)    (None, 223, 223, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 110, 110, 32)      4128      
_________________________________________________________________
activation_7 (Activation)    (None, 110, 110, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 54, 54, 64)       

In [17]:
!ls

drive  model_saved.h5  sample_data  v_dataset  v_dataset.zip


In [0]:
!chmod +x model_saved.h5

In [0]:
# !/usr/bin/env python
"""
 Copyright (c) 2018 Intel Corporation

 Licensed under the Apache License, Version 2.0 (the 'License');
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an 'AS IS' BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
"""


import os
import sys

import argparse

from pathlib import Path

import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

from keras import backend as K
from keras.models import load_model


def setKerasOptions():
    K._LEARNING_PHASE = tf.constant(0)
    K.set_learning_phase(False)
    K.set_learning_phase(0)
    K.set_image_data_format('channels_last')


def getInputParameters():
    parser = argparse.ArgumentParser()
    parser.add_argument('--input_model', '-m', required=True, type=str, help='Path to Keras model.')
    parser.add_argument('--num_outputs', '-no', required=False, type=int, help='Number of outputs. 1 by default.', default=1)

    return parser


def export_keras_to_tf(input_model, output_model, num_output):
    #print('Loading Keras model: ', input_model)
    # already giving it the model so commenting out the next line
    #keras_model = load_model(input_model)

    keras_model = input_model

    print(keras_model.summary())

    predictions = [None] * num_output
    predrediction_node_names = [None] * num_output

    for i in range(num_output):
        predrediction_node_names[i] = 'output_node' + str(i)
        predictions[i] = tf.identity(keras_model.outputs[i], name=predrediction_node_names[i])

    sess = K.get_session()

    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), predrediction_node_names)
    infer_graph = graph_util.remove_training_nodes(constant_graph) 

    graph_io.write_graph(infer_graph, '.', output_model, as_text=False)




In [0]:
input_model_name = 'model_saved.h5'
model.load_weights(input_model_name)

In [38]:
num_output = 1
output_model = 'tf_model' + '.pb'
predrediction_node_names = export_keras_to_tf(model, output_model, num_output)
print('Ouput nodes are:', predrediction_node_names)
print('Saved as TF frozen model to: ', output_model)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 223, 223, 32)      416       
_________________________________________________________________
activation_6 (Activation)    (None, 223, 223, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 110, 110, 32)      4128      
_________________________________________________________________
activation_7 (Activation)    (None, 110, 110, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 54, 54, 64)       

In [39]:
!ls

drive  model_saved.h5  sample_data  tf_model.pb  v_dataset  v_dataset.zip


In [0]:
from google.colab import files
files.download('model_saved.h5') 

In [0]:
files.download('tf_model.pb')